In [8]:
# 라이브러리 임포트 하기

import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

In [9]:
# 구글드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/My Drive/데이터파일/소상공인시장진흥공단_상가업소정보_의료기관_201909.csv', encoding='UTF-8')
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,19956873,하나산부인과,NaN,S,의료,S01,병원,S01B10,산부인과,Q86201,일반 의원,41.0,경기도,41273.0,안산시 단원구,4127360000,선부3동,4.127311e+09,선부동,4127310900210810000,1,대지,1081,NaN,경기도 안산시 단원구 선부동 1081,412733191011,경기도 안산시 단원구 달미로,10,NaN,4127310900110810000010857,산호한양아파트,경기도 안산시 단원구 달미로 10,425764.0,15236.0,NaN,NaN,NaN,126.814295,37.336344
1,20024149,타워광명내과의원,NaN,S,의료,S01,병원,S01B07,내과/외과,Q86201,일반 의원,11.0,서울특별시,11680.0,강남구,1168065600,도곡2동,1.168012e+09,도곡동,1168011800204670014,1,대지,467,14.0,서울특별시 강남구 도곡동 467-14,116804166604,서울특별시 강남구 언주로30길,39,NaN,1168011800104670014000001,NaN,서울특별시 강남구 언주로30길 39,135270.0,6292.0,NaN,4,NaN,127.053198,37.488742
2,20152277,조정현신경외과의원,NaN,S,의료,S01,병원,S01B15,신경외과,Q86201,일반 의원,41.0,경기도,41390.0,시흥시,4139059100,정왕1동,4.139013e+09,정왕동,4139013200217400001,1,대지,1740,1.0,경기도 시흥시 정왕동 1740-1,413903199049,경기도 시흥시 중심상가로,178,NaN,4139013200117400001017064,한라프라자,경기도 시흥시 중심상가로 178,429450.0,15066.0,NaN,NaN,NaN,126.734841,37.344955
3,20350610,한귀원정신과의원,NaN,S,의료,S01,병원,S01B99,기타병원,NaN,NaN,26.0,부산광역시,26500.0,수영구,2650078000,광안3동,2.650010e+09,광안동,2650010400200740001,1,대지,74,1.0,부산광역시 수영구 광안동 74-1,265003131037,부산광역시 수영구 수영로,688,NaN,2650010400100740001009932,NaN,부산광역시 수영구 수영로 688,613100.0,48266.0,NaN,5,NaN,129.115438,35.166872
4,20364049,더블유스토어수지점,수지점,S,의료,S02,약국/한약방,S02A01,약국,G47811,의약품 및 의료용품 소매업,41.0,경기도,41465.0,용인시 수지구,4146551000,풍덕천1동,4.146510e+09,풍덕천동,4146510100207120002,1,대지,712,2.0,경기도 용인시 수지구 풍덕천동 712-2,414653205024,경기도 용인시 수지구 문정로,32,NaN,4146510100107120002026238,NaN,경기도 용인시 수지구 문정로 32,448170.0,16837.0,NaN,1,NaN,127.095522,37.323528


# 1. 단순회귀분석

In [ ]:
# smf.ols = 최소제곱법으로 회귀모형을 만듦, formula = '종속변수 ~ 독립변수'

model = smf.ols(formula= '상가업소번호 ~ 경도', data=data)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 상가업소번호   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     29.34
Date:                Fri, 23 Oct 2020   Prob (F-statistic):           6.09e-08
Time:                        15:46:59   Log-Likelihood:            -1.5392e+06
No. Observations:               91335   AIC:                         3.078e+06
Df Residuals:                   91333   BIC:                         3.078e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    7.55e+06   2.52e+06      2.992      0.003     2.6e+06    1.25e+07
경도          1.072e+05   1.98e+04      5.417      0.000    6.84e+04    1.46e+05
==============================================================================
Omnibus:                     8335.774   Durbin-Watson:                   0.767
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10803.478
Skew:                          -0.839   Prob(JB):                         0.00
Kurtosis:                       2.851   Cond. No.                     1.93e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.93e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 2. 다중회귀분석

다중공선성이 존재하는 것을 파악했다면 우리는 어떻게 다중공선성 문제를 해결할 수 있을까? 다중공선성 문제해결 방법은 다음과 같다.

              첫번째, 다중공선성이 의심되는 변수를 제거한다. 

              두번째, 독립변수 입력방식을 바꿔본다.

              세번째, 독립변수들을 요인으로 묶어 분석을 한다.

In [ ]:
data.columns

model = smf.ols(formula= '상가업소번호 ~ 경도 + 시군구코드 + 시도코드', data=data)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 상가업소번호   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     13.85
Date:                Fri, 23 Oct 2020   Prob (F-statistic):           5.04e-09
Time:                        15:47:21   Log-Likelihood:            -1.5327e+06
No. Observations:               90956   AIC:                         3.066e+06
Df Residuals:                   90952   BIC:                         3.066e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.891e+06   2.55e+06      2.699      0.007    1.89e+06    1.19e+07
경도          1.139e+05      2e+04      5.680      0.000    7.46e+04    1.53e+05
시군구코드       -118.1825     83.463     -1.416      0.157    -281.769      45.404
시도코드        1.135e+05   8.31e+04      1.366      0.172   -4.93e+04    2.76e+05
==============================================================================
Omnibus:                     8290.880   Durbin-Watson:                   0.765
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10739.584
Skew:                          -0.838   Prob(JB):                         0.00
Kurtosis:                       2.850   Cond. No.                     5.41e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.41e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 다중공선성 확인 - VIF

In [ ]:
y, X = dmatrices("상가업소번호 ~" + features, data=data, return_type = "dataframe")
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif.round(1)

NameError: ignored